# Prep

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
% cd drive/MyDrive/sp/data
! ls

/content/drive/MyDrive/sp/data
FakeData_EPL.csv   KaggleDataset_withBO.txt  PL_scraped_ord.csv
KaggleDataset.csv  old_FakeData_EPL.csv


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from torch.nn import Embedding,\
                     Module,\
                     Linear,\
                     Dropout,\
                     Tanh,\
                     ReLU,\
                     BatchNorm1d,\
                     LogSoftmax,\
                     NLLLoss,\
                     Conv1d,\
                     ModuleList
import torch.nn.functional as F

# Dataset

In [4]:
dataset = pd.read_csv('KaggleDataset_withBO.txt')
dataset.tail(3)

,match_id,country,league,season,week,date,home_team,away_team,home_goal,away_goal,result,home_lineup,away_lineup,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA
19524,24495,Spain,Spain LIGA BBVA,2015/2016,38,2016-05-15 00:00:00,Málaga CF,UD Las Palmas,4,1,win,Guillermo Ochoa - Miguel Torres - Raul Albento...,Raul Lizoain - David Garcia - Pedro Bigas - Pa...,1.80,3.75,4.5,1.83,3.70,4.00,1.85,3.45,4.0,1.80,3.60,4.33
19525,24496,Spain,Spain LIGA BBVA,2015/2016,38,2016-05-14 00:00:00,Atlético Madrid,RC Celta de Vigo,2,0,win,Jan Oblak - Juanfran - Stefan Savic - Diego Go...,Sergio Alvarez - Johny - Hugo Mallo - Sergi Go...,1.75,3.75,4.5,1.83,3.60,4.10,1.85,3.70,3.7,1.83,3.60,4.20
19526,24497,Spain,Spain LIGA BBVA,2015/2016,38,2016-05-15 00:00:00,Rayo Vallecano,Levante UD,3,1,win,Yoel Rodriguez - Quini - Antonio Amaya - Tito ...,Diego Marino - Ivan Lopez - David Navarro - Ca...,1.33,5.25,9.0,1.33,4.75,9.25,1.40,5.00,6.0,1.33,5.25,9.00


# Dataset Transform

## Label Setting

In [5]:
teams = np.unique(dataset[['away_team', 'home_team']].values)
team_lblenc = LabelEncoder()
team_lblenc.fit(teams)

players = np.unique(
    np.concatenate(
      (
        np.stack(dataset['home_lineup'].apply(lambda lineup: lineup.split(' - '))).reshape(-1),
        np.stack(dataset['away_lineup'].apply(lambda lineup: lineup.split(' - '))).reshape(-1)
      )    
    )
)
player_lblenc = LabelEncoder()
player_lblenc.fit(players)

results = pd.unique(dataset['result'])
result_lblenc = LabelEncoder()
result_lblenc.fit(results)

LabelEncoder()

## Labeling

In [6]:
home_team_labels = team_lblenc.transform(dataset['home_team'])
away_team_labels = team_lblenc.transform(dataset['away_team'])

home_player_labels = player_lblenc.transform(
     np.stack(dataset['home_lineup'].apply(lambda lineup: lineup.split(' - '))).reshape(-1)
).reshape(-1, 11)
away_player_labels = player_lblenc.transform(
     np.stack(dataset['away_lineup'].apply(lambda lineup: lineup.split(' - '))).reshape(-1)
).reshape(-1, 11)

result_labels = result_lblenc.transform(dataset['result'])



In [7]:
str_home_players = np.char.mod('%d', home_player_labels)
str_home_player_labels = np.apply_along_axis(lambda arr: ' - '.join(arr), axis=1, arr=str_home_players)

str_away_players = np.char.mod('%d', away_player_labels)
str_away_player_labels = np.apply_along_axis(lambda arr: ' - '.join(arr), axis=1, arr=str_away_players)

## Creating dataset

In [8]:
meta_dict = {
        'home_team_label': home_team_labels,
        'away_team_label': away_team_labels,
        'result_label': result_labels,
        'home_lineup_label': str_home_player_labels,
        'away_lineup_label': str_away_player_labels,
}

for column in dataset.columns:
  meta_dict.update({column: dataset[column].values})

transformed_dataset = pd.DataFrame(
    meta_dict
)

In [9]:
transformed_dataset.head()

,home_team_label,away_team_label,result_label,home_lineup_label,away_lineup_label,match_id,country,league,season,week,date,home_team,away_team,home_goal,away_goal,result,home_lineup,away_lineup,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA
0,120,116,2,9306 - 4986 - 6709 - 6262 - 4819 - 5142 - 4320...,2135 - 4335 - 2309 - 2051 - 679 - 8929 - 1874 ...,146,Belgium,Belgium Jupiler League,2008/2009,24,2009-02-27 00:00:00,KV Mechelen,KRC Genk,2,1,win,Wouter Biebauw - Kenny van Hoevelen - Nana Asa...,Davino Verhulst - Joao Carlos - Dimitri Daesel...,3.00,3.4,2.30,2.95,3.25,2.20,2.80,3.1,2.2,2.75,3.25,2.20
1,117,45,0,1161 - 2200 - 730 - 3114 - 9195 - 8770 - 8323 ...,8645 - 6361 - 747 - 4241 - 9052 - 5108 - 4903 ...,154,Belgium,Belgium Jupiler League,2008/2009,25,2009-03-08 00:00:00,KSV Cercle Brugge,Club Brugge KV,1,3,loss,Bram Verbist - Denis Viane - Anthony Portier -...,Stijn Stijnen - Michael Klukowski - Antolin Al...,2.80,3.2,2.37,3.10,3.00,2.20,2.80,3.1,2.2,3.20,3.20,2.00
2,173,203,2,2140 - 7005 - 835 - 5777 - 7839 - 4021 - 5468 ...,8097 - 4902 - 8643 - 956 - 4228 - 5515 - 5041 ...,156,Belgium,Belgium Jupiler League,2008/2009,25,2009-03-07 00:00:00,RSC Anderlecht,SV Zulte-Waregem,2,0,win,Davy Schollen - Olivier Deschacht - Arnold Kru...,Sammy Bossuyt - Karel D'Haene - Stijn Minne - ...,1.40,4.5,8.00,1.35,4.00,9.00,1.35,4.0,7.0,1.36,4.00,7.00
3,120,173,2,9306 - 4986 - 6709 - 9310 - 5109 - 4819 - 6262...,2140 - 7005 - 7839 - 6758 - 9109 - 4021 - 4515...,163,Belgium,Belgium Jupiler League,2008/2009,26,2009-03-13 00:00:00,KV Mechelen,RSC Anderlecht,2,1,win,Wouter Biebauw - Kenny van Hoevelen - Nana Asa...,Davy Schollen - Olivier Deschacht - Roland Juh...,4.50,3.6,1.75,4.70,3.30,1.70,4.20,3.2,1.7,4.00,3.40,1.72
4,203,118,1,8097 - 4902 - 8580 - 8643 - 956 - 5515 - 5041 ...,4858 - 1796 - 6579 - 4218 - 740 - 859 - 8489 -...,169,Belgium,Belgium Jupiler League,2008/2009,26,2009-03-14 00:00:00,SV Zulte-Waregem,KSV Roeselare,0,0,tie,Sammy Bossuyt - Karel D'Haene - Steve Colpaert...,Jurgen Sierens - Damir Mirvic - Mladen Lazarev...,1.75,3.5,4.75,1.60,3.55,5.05,1.65,3.4,4.2,1.61,3.25,5.00


# Dataset Split

In [10]:
info = """
Belgium Jupiler League
	 630
	 273
	 278
England Premier League
	 1749
	 605
	 600
France Ligue 1
	 1700
	 590
	 565
Germany 1. Bundesliga
	 1392
	 484
	 492
Italy Serie A
	 1609
	 552
	 543
Netherlands Eredivisie
	 1161
	 440
	 418
Portugal Liga ZON Sagres
	 627
	 280
	 323
Scotland Premier League
	 897
	 317
	 315
Spain LIGA BBVA
	 1538
	 585
	 564
"""
leagues = []
splits = []
for i in info.split('\n'):
  j = i.strip()
  if any([c.isalpha() for c in j]):
    leagues.append(j)
  elif j:
    splits.append(j)

leagues = np.array(leagues).tolist()
sizes = np.array(splits).reshape(-1, 3).astype(np.int16).tolist()
splits = dict(zip(leagues, sizes))
splits

{'Belgium Jupiler League': [630, 273, 278],
 'England Premier League': [1749, 605, 600],
 'France Ligue 1': [1700, 590, 565],
 'Germany 1. Bundesliga': [1392, 484, 492],
 'Italy Serie A': [1609, 552, 543],
 'Netherlands Eredivisie': [1161, 440, 418],
 'Portugal Liga ZON Sagres': [627, 280, 323],
 'Scotland Premier League': [897, 317, 315],
 'Spain LIGA BBVA': [1538, 585, 564]}

In [12]:
train_groups = []
dev_groups = []
test_groups = []
# batch_size = 64
for league, data in transformed_dataset.groupby('league'):
  # split_size = int(data.shape[0] * 0.63 // batch_size * batch_size)
  # split_size = int(data.shape[0] * 0.64)
  split_size = splits[league][0]
  train_set, dev_test = train_test_split(data, shuffle=False, stratify=None, train_size=split_size)
  split_size = splits[league][1]
  dev_set, test_set = train_test_split(dev_test, shuffle=False, stratify=None, train_size=0.5)

  print(league)
  print('Train Dev Test')
  print(train_set.shape[0], dev_set.shape[0], test_set.shape[0])
  print(f'{train_set.shape[0] / data.shape[0] * 100:.2f}%, {dev_set.shape[0] / data.shape[0] * 100:.2f}%, {test_set.shape[0] / data.shape[0] * 100:.2f}%')
  print('-' * 24)

  train_groups.append(train_set)
  dev_groups.append(dev_set)
  test_groups.append(test_set)

Belgium Jupiler League
Train Dev Test
630 275 276
53.34%, 23.29%, 23.37%
------------------------
England Premier League
Train Dev Test
1749 602 603
59.21%, 20.38%, 20.41%
------------------------
France Ligue 1
Train Dev Test
1700 577 578
59.54%, 20.21%, 20.25%
------------------------
Germany 1. Bundesliga
Train Dev Test
1392 488 488
58.78%, 20.61%, 20.61%
------------------------
Italy Serie A
Train Dev Test
1609 547 548
59.50%, 20.23%, 20.27%
------------------------
Netherlands Eredivisie
Train Dev Test
1161 429 429
57.50%, 21.25%, 21.25%
------------------------
Portugal Liga ZON Sagres
Train Dev Test
627 301 302
50.98%, 24.47%, 24.55%
------------------------
Scotland Premier League
Train Dev Test
897 316 316
58.67%, 20.67%, 20.67%
------------------------
Spain LIGA BBVA
Train Dev Test
1538 574 575
57.24%, 21.36%, 21.40%
------------------------


# Optimization and Evaluation Algorithms

## Training

In [64]:
def train(model, optimizer, home, away, y, loss_fn):
  model.train()
  optimizer.zero_grad()

  output = model(home, away) # forward prop
  loss = loss_fn(output, y) # loss
  loss.backward() # backprop
  optimizer.step() # parameter updating

  return loss.item()

## Evaluating

In [65]:
@torch.no_grad()
def evaluate(model, home, away, y):
  model.eval()

  output = model(home, away)
  prediction = output.argmax(-1)

  correct = (prediction == y).sum()

  return correct.item(), y.shape[0]

## Gradient Descent Fitting

In [73]:
def fit(model, train_set, batch_size, dev_set, optimizer, loss_fn, num_epochs, every=100):
  # assert train_set[0].shape[-1] == train_set[1].shape[-1]
  # assert dev_set[0].shape[-1] == dev_set[1].shape[-1]
  
  train_home, train_away, train_y = train_set
  train_home_batches = torch.split(train_home, batch_size)
  
  train_away_batches = torch.split(train_away, batch_size)
  train_y_batches = torch.split(train_y, batch_size)

  dev_home, dev_away, dev_y = dev_set

  print('Initial State')
  train_correct, train_all = evaluate(
      model, 
      train_home, 
      train_away, 
      train_y
  )
  dev_correct, dev_all = evaluate(model, dev_home, dev_away, dev_y)
  print(f'Train Acc%: {train_correct / train_all * 100:.4f}')
  print(f'Dev   Acc%: {dev_correct / dev_all * 100:.4f}')

  for epoch in range(1, num_epochs + 1):
    
    epoch_loss = 0
    for home_batch, away_batch, y_batch in zip(train_home_batches, train_away_batches, train_y_batches):
      if home_batch.shape[0] == 1:
        continue
      batch_loss = train(model, optimizer, home_batch, away_batch, y_batch, loss_fn)
      epoch_loss += batch_loss
    train_correct, train_all = evaluate(
      model, 
      train_home, 
      train_away, 
      train_y
    )
    dev_correct, dev_all = evaluate(model, dev_home, dev_away, dev_y)
    if epoch % every == 0:
      print('-' * 60)
      print(f'Epoch {epoch}')
      print(f'Avg Train Loss: {epoch_loss / train_home.shape[0]:.4f}')
      print(f'Train Acc%:     {train_correct / train_all * 100:.4f}')
      print(f'Dev   Acc%:     {dev_correct / dev_all * 100:.4f}')

## Main

In [67]:
def main(train_group, batch_size, dev_group, test_group, league, model, optimizer, loss_fn, n_epochs, every):
  try:
    (home_train, away_train, y_train) = train_group
    (home_dev, away_dev, y_dev) = dev_group
    (home_test, away_test, y_test) = test_group
    print(f'Fitting on the {league} for {n_epochs} epochs')
    print('.' * 60)
    fit(
      model, 
      (home_train, away_train, y_train),
      batch_size,
      (home_dev, away_dev, y_dev),
      optimizer,
      loss_fn,
      n_epochs,
      every
    )
  except KeyboardInterrupt:
      pass
  finally:
    test_correct, test_all = evaluate(model, home_test, away_test, y_test.reshape(-1))
    print(f'Test Acc%: {test_correct / test_all * 100:.4f}')
    print('=' * 60)


# Blade Chest Model

In [158]:
class BladeChest(Module):
  def __init__(self, feature_size, blade_chest_size, dropout_p=0.5):
    super(BladeChest, self).__init__()
    self.feature_size = feature_size
    self.blade_chest_size = blade_chest_size
    self.dropout_p = dropout_p
    self.chest_transform = Linear(self.feature_size, self.blade_chest_size, bias=False)
    self.chest_bn = BatchNorm1d(self.blade_chest_size)

    self.blade_transform = Linear(self.feature_size, self.blade_chest_size, bias=False)
    self.blade_bn = BatchNorm1d(self.blade_chest_size)

    self.regularizer = Dropout(p=self.dropout_p)
    self.activation = Tanh()

    self.result_transform = Linear(1, 3)
    self.classifier = LogSoftmax(dim=-1)

  def _encode_team(self, team):
    blade = self.blade_transform(team)
    blade = self.blade_bn(blade)
    blade = self.activation(blade)
    blade = self.regularizer(blade)

    chest = self.chest_transform(team)
    chest = self.chest_bn(chest)
    chest = self.activation(chest)
    chest = self.regularizer(chest)

    return blade, chest


  def _matchup(self, home_blade, home_chest, away_blade, away_chest):
    return (home_blade * away_chest).sum(-1) - (away_blade * home_chest).sum(-1)

  def forward(self, home, away):
    home_blade, home_chest = self._encode_team(home)
    away_blade, away_chest = self._encode_team(away)

    matchup_score = self._matchup(home_blade, home_chest, away_blade, away_chest).reshape(-1, 1)

    result = self.result_transform(matchup_score)
    result = self.classifier(result)
    result = self.regularizer(result)

    return result

# Cat Feed Forward Model

In [185]:
class CatFeedForward(Module):
  def __init__(self, feature_size, hidden_sizes, dropout_p=0.5):
    super(CatFeedForward, self).__init__()
    self.feature_size = feature_size
    self.hidden_sizes = hidden_sizes
    self.dropout_p = dropout_p

    layers_list = [feature_size * 2] + self.hidden_sizes + [3]

    self.fc_layers = ModuleList(
        [
         Linear(size, layers_list[index + 1])
         for index, size in enumerate(layers_list[:-1])
        ]
    )

    self.bn_layers = ModuleList(
        [
         BatchNorm1d(size)
         for size in layers_list[1:-1]
        ]
    )

    self.regularizer = Dropout(p=self.dropout_p)
    self.activation = ReLU()

    self.classifier = LogSoftmax(dim=-1)

  def forward(self, home, away):
    h = torch.cat((home, away), dim=-1)
    for fc, bn in zip(self.fc_layers[:-1], self.bn_layers):
      h = fc(h)
      h = bn(h)
      h = self.activation(h)
      h = self.regularizer(h)
      
    result = self.fc_layers[-1](h)
    result = self.classifier(result)
    result = self.regularizer(result)

    return result

# Team Blade Chest Modeling

## TeamBladeChest Model 

In [32]:
class TeamBladeChest(Module):
  def __init__(self, num_teams, embedding_size, blade_chest_size, dropout=0.5):
    super(TeamBladeChest, self).__init__()
    self.num_teams = num_teams
    self.embedding_size = embedding_size
    self.blade_chest_size = blade_chest_size
    self.dropout=dropout
    self.team_embedder = Embedding(self.num_teams, self.embedding_size)
    self.emb_bn = BatchNorm1d(self.embedding_size)
    self.decoder = BladeChest(self.embedding_size, self.blade_chest_size)
    self.regularizer = Dropout(p=self.dropout)

  def _encode_team(self, team):
    embedding = self.team_embedder(team)
    embedding = self.emb_bn(embedding)
    #dropout here?
    emnedding = self.regularizer(embedding)
    return embedding

  def forward(self, home, away):
    home_encoded = self._encode_team(home)
    away_encoded = self._encode_team(away)
    return self.decoder(home_encoded, away_encoded)

## Hyperparameters

In [165]:
home = torch.from_numpy(transformed_dataset.loc[:, ['home_team_label']].values.reshape(-1))
away = torch.from_numpy(transformed_dataset.loc[:, ['away_team_label']].values.reshape(-1))
y = torch.from_numpy(transformed_dataset.loc[:, ['result_label']].values.reshape(-1))

assert home.max() == away.max()
num_teams = home.max() + 1
embedding_size = 4
hidden_size = 8
batch_size = 32
learning_rate = 1e-3
n_epochs = 250
every = 25

tbc_model = TeamBladeChest(
    num_teams,
    embedding_size,
    hidden_size,
    dropout=0.5
)

optimizer = torch.optim.Adam(tbc_model.parameters(), lr=learning_rate)
criterion = NLLLoss()
tbc_model

TeamBladeChest(
  (team_embedder): Embedding(254, 4)
  (emb_bn): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (decoder): BladeChest(
    (chest_transform): Linear(in_features=4, out_features=8, bias=False)
    (chest_bn): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (blade_transform): Linear(in_features=4, out_features=8, bias=False)
    (blade_bn): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (regularizer): Dropout(p=0.5, inplace=False)
    (activation): Tanh()
    (result_transform): Linear(in_features=1, out_features=3, bias=True)
    (classifier): LogSoftmax(dim=-1)
  )
  (regularizer): Dropout(p=0.5, inplace=False)
)

## Fitting

In [166]:
for train_set, dev_set, test_set in zip(train_groups, dev_groups, test_groups):
  # batch_size = train_set.shape[0]
  home_train = torch.from_numpy(train_set.loc[:, ['home_team_label']].values.reshape(-1))
  # home_train = torch.split(home_train, batch_size)
  away_train = torch.from_numpy(train_set.loc[:, ['away_team_label']].values.reshape(-1))
  # away_train = torch.split(away_train, batch_size)
  y_train = torch.from_numpy(train_set.loc[:, ['result_label']].values.reshape(-1))
  # y_train = torch.split(y_train, batch_size)

  home_dev = torch.from_numpy(dev_set.loc[:, ['home_team_label']].values.reshape(-1))
  away_dev = torch.from_numpy(dev_set.loc[:, ['away_team_label']].values.reshape(-1))
  y_dev = torch.from_numpy(dev_set.loc[:, ['result_label']].values.reshape(-1))

  home_test = torch.from_numpy(test_set.loc[:, ['home_team_label']].values.reshape(-1))
  away_test = torch.from_numpy(test_set.loc[:, ['away_team_label']].values.reshape(-1))
  y_test = torch.from_numpy(test_set.loc[:, ['result_label']].values.reshape(-1))

  main(
      (home_train, away_train, y_train),
       batch_size,
      (home_dev, away_dev, y_dev),
      (home_test, away_test, y_test),
      np.unique(train_set["league"].values).item(),
      tbc_model, 
      optimizer,
       criterion,
       n_epochs,
       every    
  )

Fitting on the Belgium Jupiler League for 250 epochs
............................................................
Initial State
Train Acc%: 25.7143
Dev   Acc%: 24.7273
------------------------------------------------------------
Epoch 25
Avg Train Loss: 0.0342
Train Acc%:     48.0952
Dev   Acc%:     42.1818
------------------------------------------------------------
Epoch 50
Avg Train Loss: 0.0334
Train Acc%:     48.0952
Dev   Acc%:     42.1818
------------------------------------------------------------
Epoch 75
Avg Train Loss: 0.0336
Train Acc%:     48.0952
Dev   Acc%:     42.1818
------------------------------------------------------------
Epoch 100
Avg Train Loss: 0.0328
Train Acc%:     48.2540
Dev   Acc%:     41.8182
------------------------------------------------------------
Epoch 125
Avg Train Loss: 0.0329
Train Acc%:     48.5714
Dev   Acc%:     41.4545
------------------------------------------------------------
Epoch 150
Avg Train Loss: 0.0335
Train Acc%:     50.0000
Dev   A

# Team Cat Feed Forward Modeling

## TeamCatFeedForward Model 

In [174]:
class TeamCatFeedForward(Module):
  def __init__(self, num_teams, embedding_size, feed_forward_sizes, dropout=0.5):
    super(TeamCatFeedForward, self).__init__()
    self.num_teams = num_teams
    self.embedding_size = embedding_size
    self.feed_forward_sizes = feed_forward_sizes
    self.dropout=dropout
    self.team_embedder = Embedding(self.num_teams, self.embedding_size)
    self.emb_bn = BatchNorm1d(self.embedding_size)
    self.decoder = CatFeedForward(self.embedding_size, self.feed_forward_sizes)
    self.regularizer = Dropout(p=self.dropout)

  def _encode_team(self, team):
    embedding = self.team_embedder(team)
    embedding = self.emb_bn(embedding)
    #dropout here?
    emnedding = self.regularizer(embedding)
    return embedding

  def forward(self, home, away):
    home_encoded = self._encode_team(home)
    away_encoded = self._encode_team(away)
    return self.decoder(home_encoded, away_encoded)

## Hyperparameters

In [186]:
home = torch.from_numpy(transformed_dataset.loc[:, ['home_team_label']].values.reshape(-1))
away = torch.from_numpy(transformed_dataset.loc[:, ['away_team_label']].values.reshape(-1))
y = torch.from_numpy(transformed_dataset.loc[:, ['result_label']].values.reshape(-1))

assert home.max() == away.max()
num_teams = home.max() + 1
embedding_size = 4
hidden_sizes = [8]
batch_size = 32
learning_rate = 1e-3
n_epochs = 250
every = 25

tcff_model = TeamCatFeedForward(
    num_teams,
    embedding_size,
    hidden_sizes,
    dropout=0.5
)

optimizer = torch.optim.Adam(tcff_model.parameters(), lr=learning_rate)
criterion = NLLLoss()
tcff_model

TeamCatFeedForward(
  (team_embedder): Embedding(254, 4)
  (emb_bn): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (decoder): CatFeedForward(
    (fc_layers): ModuleList(
      (0): Linear(in_features=8, out_features=8, bias=True)
      (1): Linear(in_features=8, out_features=3, bias=True)
    )
    (bn_layers): ModuleList(
      (0): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (regularizer): Dropout(p=0.5, inplace=False)
    (activation): ReLU()
    (classifier): LogSoftmax(dim=-1)
  )
  (regularizer): Dropout(p=0.5, inplace=False)
)

## Fitting

In [187]:
for train_set, dev_set, test_set in zip(train_groups, dev_groups, test_groups):
  # batch_size = train_set.shape[0]
  home_train = torch.from_numpy(train_set.loc[:, ['home_team_label']].values.reshape(-1))
  # home_train = torch.split(home_train, batch_size)
  away_train = torch.from_numpy(train_set.loc[:, ['away_team_label']].values.reshape(-1))
  # away_train = torch.split(away_train, batch_size)
  y_train = torch.from_numpy(train_set.loc[:, ['result_label']].values.reshape(-1))
  # y_train = torch.split(y_train, batch_size)

  home_dev = torch.from_numpy(dev_set.loc[:, ['home_team_label']].values.reshape(-1))
  away_dev = torch.from_numpy(dev_set.loc[:, ['away_team_label']].values.reshape(-1))
  y_dev = torch.from_numpy(dev_set.loc[:, ['result_label']].values.reshape(-1))

  home_test = torch.from_numpy(test_set.loc[:, ['home_team_label']].values.reshape(-1))
  away_test = torch.from_numpy(test_set.loc[:, ['away_team_label']].values.reshape(-1))
  y_test = torch.from_numpy(test_set.loc[:, ['result_label']].values.reshape(-1))

  main(
      (home_train, away_train, y_train),
       batch_size,
      (home_dev, away_dev, y_dev),
      (home_test, away_test, y_test),
      np.unique(train_set["league"].values).item(),
      tcff_model, 
      optimizer,
       criterion,
       n_epochs,
       every    
  )

Fitting on the Belgium Jupiler League for 250 epochs
............................................................
Initial State
Train Acc%: 43.4921
Dev   Acc%: 43.2727
------------------------------------------------------------
Epoch 25
Avg Train Loss: 0.0338
Train Acc%:     49.3651
Dev   Acc%:     41.8182
------------------------------------------------------------
Epoch 50
Avg Train Loss: 0.0332
Train Acc%:     51.5873
Dev   Acc%:     42.9091
------------------------------------------------------------
Epoch 75
Avg Train Loss: 0.0333
Train Acc%:     51.1111
Dev   Acc%:     43.2727
------------------------------------------------------------
Epoch 100
Avg Train Loss: 0.0338
Train Acc%:     52.0635
Dev   Acc%:     42.9091
------------------------------------------------------------
Epoch 125
Avg Train Loss: 0.0311
Train Acc%:     52.2222
Dev   Acc%:     43.6364
------------------------------------------------------------
Epoch 150
Avg Train Loss: 0.0315
Train Acc%:     52.3810
Dev   A

# Player Blade Chest Modeling

## PlayerOneHot Model

In [37]:
class PlayerOneHot(Module):
  def __init__(self, num_players):
    super(PlayerOneHot, self).__init__()
    self.num_players = num_players

  def forward(self, players):
    return F.one_hot(players, self.num_players).sum(-2).double()

## PlayerBladeChest Model

In [109]:
class PlayerBladeChest(Module):
  def __init__(self, num_players, blade_chest_size, dropout_p=0.5):
    super(PlayerBladeChest, self).__init__()
    self.num_players = num_players
    self.blade_chest_size = blade_chest_size
    self.dropout_p = dropout_p
    self.one_hot = PlayerOneHot(self.num_players)
    self.decoder = BladeChest(self.num_players, self.blade_chest_size)

  def forward(self, home, away):
    home_encoded = self.one_hot(home)
    away_encoded = self.one_hot(away) 

    return self.decoder(home_encoded, away_encoded)

## Hyperparameters

In [113]:
home_players = torch.from_numpy(home_player_labels)
away_players = torch.from_numpy(away_player_labels)
assert home_players.max() == away_players.max()
num_players = home_players.max() + 1

# embedding_size = 5
hidden_size = 2
learning_rate = 1e-3
n_epochs = 15
every = 3
batch_size = 32

pbc_model = PlayerBladeChest(
    num_players,
    hidden_size,
    dropout_p=0.5
).double()

optimizer = torch.optim.Adam(pbc_model.parameters(), lr=learning_rate)
criterion = NLLLoss()
pbc_model

PlayerBladeChest(
  (one_hot): PlayerOneHot()
  (decoder): BladeChest(
    (chest_transform): Linear(in_features=9513, out_features=2, bias=False)
    (chest_bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (blade_transform): Linear(in_features=9513, out_features=2, bias=False)
    (blade_bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (regularizer): Dropout(p=0.5, inplace=False)
    (activation): Tanh()
    (result_transform): Linear(in_features=1, out_features=3, bias=True)
    (classifier): LogSoftmax(dim=-1)
  )
)

## Fitting

In [114]:
for train_set, dev_set, test_set in zip(train_groups, dev_groups, test_groups):
  # batch_size = train_set.shape[0]
  home_train = torch.from_numpy(
      np.stack(train_set['home_lineup_label'].apply(lambda lineup: lineup.split(' - '))).astype(np.int16)
  ).long()
  home_train = home_train.reshape(-1, 11)
  away_train = torch.from_numpy(
      np.stack(train_set['away_lineup_label'].apply(lambda lineup: lineup.split(' - '))).astype(np.int16)
  ).long()
  away_train = away_train.reshape(-1, 11)
  y_train = torch.from_numpy(train_set.loc[:, ['result_label']].values.reshape(-1))

  home_dev = torch.from_numpy(
      np.stack(dev_set['home_lineup_label'].apply(lambda lineup: lineup.split(' - '))).astype(np.int16)
  ).long()
  away_dev = torch.from_numpy(
      np.stack(dev_set['away_lineup_label'].apply(lambda lineup: lineup.split(' - '))).astype(np.int16)
  ).long()
  y_dev = torch.from_numpy(dev_set.loc[:, ['result_label']].values.reshape(-1))

  home_test = torch.from_numpy(
      np.stack(test_set['home_lineup_label'].apply(lambda lineup: lineup.split(' - '))).astype(np.int16)
  ).long()
  away_test = torch.from_numpy(
      np.stack(test_set['home_lineup_label'].apply(lambda lineup: lineup.split(' - '))).astype(np.int16)
  ).long()
  y_test = torch.from_numpy(test_set.loc[:, ['result_label']].values.reshape(-1))

  main(
      (home_train, away_train, y_train),
       batch_size,
      (home_dev, away_dev, y_dev),
      (home_test, away_test, y_test),
      np.unique(train_set["league"].values).item(),
      pbc_model, 
      optimizer,
       criterion,
       n_epochs,
       every
  )

Fitting on the Belgium Jupiler League for 15 epochs
............................................................
Initial State
Train Acc%: 48.0952
Dev   Acc%: 42.1818
------------------------------------------------------------
Epoch 3
Avg Train Loss: 0.0519
Train Acc%:     51.1111
Dev   Acc%:     38.1818
------------------------------------------------------------
Epoch 6
Avg Train Loss: 0.0508
Train Acc%:     53.3333
Dev   Acc%:     40.0000
------------------------------------------------------------
Epoch 9
Avg Train Loss: 0.0469
Train Acc%:     54.9206
Dev   Acc%:     38.9091
------------------------------------------------------------
Epoch 12
Avg Train Loss: 0.0486
Train Acc%:     56.8254
Dev   Acc%:     40.7273
------------------------------------------------------------
Epoch 15
Avg Train Loss: 0.0473
Train Acc%:     58.8889
Dev   Acc%:     40.0000
Test Acc%: 47.4638
Fitting on the England Premier League for 15 epochs
............................................................

# Player Cat Feed Forwards Modeling

## PlayerCatFeedForward Model

In [188]:
class PlayerCatFeedForward(Module):
  def __init__(self, num_players, feed_forward_sizes, dropout_p=0.5):
    super(PlayerCatFeedForward, self).__init__()
    self.num_players = num_players
    self.dropout_p = dropout_p
    self.feed_forward_sizes = feed_forward_sizes
    self.one_hot = PlayerOneHot(self.num_players)
    self.decoder = CatFeedForward(self.num_players, self.feed_forward_sizes)

  def forward(self, home, away):
    home_encoded = self.one_hot(home)
    away_encoded = self.one_hot(away) 

    return self.decoder(home_encoded, away_encoded)

## Hyperparameters

In [192]:
home_players = torch.from_numpy(home_player_labels)
away_players = torch.from_numpy(away_player_labels)
assert home_players.max() == away_players.max()
num_players = home_players.max() + 1

# embedding_size = 5
hidden_sizes = [2]
learning_rate = 1e-3
n_epochs = 10
every = 2
batch_size = 32

pcff_model = PlayerCatFeedForward(
    num_players,
    hidden_sizes,
    dropout_p=0.5
).double()

optimizer = torch.optim.Adam(pcff_model.parameters(), lr=learning_rate)
criterion = NLLLoss()
pcff_model

PlayerCatFeedForward(
  (one_hot): PlayerOneHot()
  (decoder): CatFeedForward(
    (fc_layers): ModuleList(
      (0): Linear(in_features=19026, out_features=2, bias=True)
      (1): Linear(in_features=2, out_features=3, bias=True)
    )
    (bn_layers): ModuleList(
      (0): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (regularizer): Dropout(p=0.5, inplace=False)
    (activation): ReLU()
    (classifier): LogSoftmax(dim=-1)
  )
)

## Fitting

In [191]:
for train_set, dev_set, test_set in zip(train_groups, dev_groups, test_groups):
  # batch_size = train_set.shape[0]
  home_train = torch.from_numpy(
      np.stack(train_set['home_lineup_label'].apply(lambda lineup: lineup.split(' - '))).astype(np.int16)
  ).long()
  home_train = home_train.reshape(-1, 11)
  away_train = torch.from_numpy(
      np.stack(train_set['away_lineup_label'].apply(lambda lineup: lineup.split(' - '))).astype(np.int16)
  ).long()
  away_train = away_train.reshape(-1, 11)
  y_train = torch.from_numpy(train_set.loc[:, ['result_label']].values.reshape(-1))

  home_dev = torch.from_numpy(
      np.stack(dev_set['home_lineup_label'].apply(lambda lineup: lineup.split(' - '))).astype(np.int16)
  ).long()
  away_dev = torch.from_numpy(
      np.stack(dev_set['away_lineup_label'].apply(lambda lineup: lineup.split(' - '))).astype(np.int16)
  ).long()
  y_dev = torch.from_numpy(dev_set.loc[:, ['result_label']].values.reshape(-1))

  home_test = torch.from_numpy(
      np.stack(test_set['home_lineup_label'].apply(lambda lineup: lineup.split(' - '))).astype(np.int16)
  ).long()
  away_test = torch.from_numpy(
      np.stack(test_set['home_lineup_label'].apply(lambda lineup: lineup.split(' - '))).astype(np.int16)
  ).long()
  y_test = torch.from_numpy(test_set.loc[:, ['result_label']].values.reshape(-1))

  main(
      (home_train, away_train, y_train),
       batch_size,
      (home_dev, away_dev, y_dev),
      (home_test, away_test, y_test),
      np.unique(train_set["league"].values).item(),
      pcff_model, 
      optimizer,
       criterion,
       n_epochs,
       every
  )

Fitting on the Belgium Jupiler League for 15 epochs
............................................................
Initial State
Train Acc%: 26.1905
Dev   Acc%: 33.0909
------------------------------------------------------------
Epoch 3
Avg Train Loss: 0.0314
Train Acc%:     48.4127
Dev   Acc%:     37.4545
------------------------------------------------------------
Epoch 6
Avg Train Loss: 0.0295
Train Acc%:     55.8730
Dev   Acc%:     37.0909
------------------------------------------------------------
Epoch 9
Avg Train Loss: 0.0300
Train Acc%:     56.9841
Dev   Acc%:     38.9091
------------------------------------------------------------
Epoch 12
Avg Train Loss: 0.0281
Train Acc%:     59.0476
Dev   Acc%:     40.3636
------------------------------------------------------------
Epoch 15
Avg Train Loss: 0.0298
Train Acc%:     60.9524
Dev   Acc%:     40.3636
Test Acc%: 46.3768
Fitting on the England Premier League for 15 epochs
............................................................

# DeepSet Blade Chest  Modeling

## DeepSetBladeChest Model

In [115]:
class DeepSetBladeChest(Module):
  def __init__(self, num_players, embedding_size, player_hidden_size, team_hidden_sizes, blade_chest_size, dropout=0.5):
    super(DeepSetBladeChest, self).__init__()
    self.num_players = num_players
    self.embedding_size = embedding_size
    self.player_hidden_size = player_hidden_size
    self.team_hidden_sizes = team_hidden_sizes
    self.blade_chest_size = blade_chest_size
    self.dropout=dropout

    self.player_embedder = Embedding(self.num_players, self.embedding_size)
    self.emb_bn = BatchNorm1d(self.embedding_size)

    self.player_fc = Conv1d(self.embedding_size, self.player_hidden_size, 1)
    self.player_fc_bn = BatchNorm1d(self.player_hidden_size)
    self.team_fc_list = [self.player_hidden_size] + list(team_hidden_sizes)
    self.team_fc = ModuleList(
        [
          Linear(size, self.team_fc_list[index + 1])
          for index, size in enumerate(self.team_fc_list[:-1])
        ]
    )
    self.team_fc_bn = ModuleList(
        [
          BatchNorm1d(size)
          for size in self.team_fc_list[1:]
        ]
    )

    self.decoder = BladeChest(self.team_fc_list[-1], self.blade_chest_size)
    self.regularizer = Dropout(p=self.dropout)
    self.fc_activation = ReLU()

  def _encode_team(self, team):
    embedding = self.player_embedder(team)
    embedding = self.emb_bn(embedding.moveaxis(-2, -1))
    # #dropout here?
    emnedding = self.regularizer(embedding)

    player_h = self.player_fc(embedding)
    player_h = self.player_fc_bn(player_h)
    player_h = self.fc_activation(player_h)
    player_h = self.regularizer(player_h)
    team_h = player_h.sum(-1)

    for fc, bn in zip(self.team_fc, self.team_fc_bn):
      team_h = fc(team_h)
      team_h = bn(team_h)
      team_h = self.fc_activation(team_h)
      team_h = self.regularizer(team_h)

    return team_h

  def forward(self, home, away):
    home_encoded = self._encode_team(home)
    away_encoded = self._encode_team(away)
    return self.decoder(home_encoded, away_encoded)

## Hyperparameters

In [126]:
home_players = torch.from_numpy(home_player_labels)
away_players = torch.from_numpy(away_player_labels)
assert home_players.max() == away_players.max()
num_players = home_players.max() + 1

embedding_size = 4
player_hidden_size = 12
team_hidden_size = [14, 16]
blade_chest_size = 16
learning_rate = 1e-3
dropout = 0.5
n_epochs = 75
every = 15
batch_size = 32

dsbc_model = DeepSetBladeChest(
    num_players, 
    embedding_size, 
    player_hidden_size, 
    team_hidden_size, 
    blade_chest_size,
    dropout=dropout
)
optimizer = torch.optim.Adam(dsbc_model.parameters(), lr=learning_rate)
criterion = NLLLoss()
dsbc_model

DeepSetBladeChest(
  (player_embedder): Embedding(9513, 4)
  (emb_bn): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (player_fc): Conv1d(4, 12, kernel_size=(1,), stride=(1,))
  (player_fc_bn): BatchNorm1d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (team_fc): ModuleList(
    (0): Linear(in_features=12, out_features=14, bias=True)
    (1): Linear(in_features=14, out_features=16, bias=True)
  )
  (team_fc_bn): ModuleList(
    (0): BatchNorm1d(14, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (decoder): BladeChest(
    (chest_transform): Linear(in_features=16, out_features=16, bias=False)
    (chest_bn): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (blade_transform): Linear(in_features=16, out_features=16, bias=False)
    (blade_bn): BatchNorm1d(16, eps=1e-05, momentum=0.1, affin

## Fitting

In [127]:
for train_set, dev_set, test_set in zip(train_groups, dev_groups, test_groups):
  # batch_size = train_set.shape[0]
  home_train = torch.from_numpy(
      np.stack(train_set['home_lineup_label'].apply(lambda lineup: lineup.split(' - '))).astype(np.int16)
  ).long()
  home_train = home_train.reshape(-1, 11)
  away_train = torch.from_numpy(
      np.stack(train_set['away_lineup_label'].apply(lambda lineup: lineup.split(' - '))).astype(np.int16)
  ).long()
  away_train = away_train.reshape(-1, 11)
  y_train = torch.from_numpy(train_set.loc[:, ['result_label']].values.reshape(-1))
  # y_train = y_train.reshape(-1, batch_size)

  home_dev = torch.from_numpy(
      np.stack(dev_set['home_lineup_label'].apply(lambda lineup: lineup.split(' - '))).astype(np.int16)
  ).long()
  away_dev = torch.from_numpy(
      np.stack(dev_set['away_lineup_label'].apply(lambda lineup: lineup.split(' - '))).astype(np.int16)
  ).long()
  y_dev = torch.from_numpy(dev_set.loc[:, ['result_label']].values.reshape(-1))

  home_test = torch.from_numpy(
      np.stack(test_set['home_lineup_label'].apply(lambda lineup: lineup.split(' - '))).astype(np.int16)
  ).long()
  away_test = torch.from_numpy(
      np.stack(test_set['home_lineup_label'].apply(lambda lineup: lineup.split(' - '))).astype(np.int16)
  ).long()
  y_test = torch.from_numpy(test_set.loc[:, ['result_label']].values.reshape(-1))

  main(
      (home_train, away_train, y_train),
       batch_size,
      (home_dev, away_dev, y_dev),
      (home_test, away_test, y_test),
      np.unique(train_set["league"].values).item(),
      dsbc_model, 
      optimizer,
       criterion,
       n_epochs,
       every
  )

Fitting on the Belgium Jupiler League for 75 epochs
............................................................
Initial State
Train Acc%: 26.1905
Dev   Acc%: 33.0909
------------------------------------------------------------
Epoch 15
Avg Train Loss: 0.0517
Train Acc%:     26.1905
Dev   Acc%:     33.0909
------------------------------------------------------------
Epoch 30
Avg Train Loss: 0.0479
Train Acc%:     26.1905
Dev   Acc%:     33.0909
------------------------------------------------------------
Epoch 45
Avg Train Loss: 0.0469
Train Acc%:     48.0952
Dev   Acc%:     42.1818
------------------------------------------------------------
Epoch 60
Avg Train Loss: 0.0448
Train Acc%:     48.0952
Dev   Acc%:     42.1818
------------------------------------------------------------
Epoch 75
Avg Train Loss: 0.0453
Train Acc%:     48.0952
Dev   Acc%:     42.1818
Test Acc%: 47.4638
Fitting on the England Premier League for 75 epochs
.........................................................

# DeepSet Blade Cat Feed Forward  Modeling

## DeepSetCatFeedForward Model

In [195]:
class DeepSetCatFeedForward(Module):
  def __init__(self, num_players, embedding_size, player_hidden_size, team_hidden_sizes, feed_forward_sizes, dropout=0.5):
    super(DeepSetCatFeedForward, self).__init__()
    self.num_players = num_players
    self.embedding_size = embedding_size
    self.player_hidden_size = player_hidden_size
    self.team_hidden_sizes = team_hidden_sizes
    self.feed_forward_sizes = feed_forward_sizes
    self.dropout=dropout

    self.player_embedder = Embedding(self.num_players, self.embedding_size)
    self.emb_bn = BatchNorm1d(self.embedding_size)

    self.player_fc = Conv1d(self.embedding_size, self.player_hidden_size, 1)
    self.player_fc_bn = BatchNorm1d(self.player_hidden_size)
    self.team_fc_list = [self.player_hidden_size] + list(team_hidden_sizes)
    self.team_fc = ModuleList(
        [
          Linear(size, self.team_fc_list[index + 1])
          for index, size in enumerate(self.team_fc_list[:-1])
        ]
    )
    self.team_fc_bn = ModuleList(
        [
          BatchNorm1d(size)
          for size in self.team_fc_list[1:]
        ]
    )

    self.decoder = CatFeedForward(self.team_fc_list[-1], self.feed_forward_sizes)
    self.regularizer = Dropout(p=self.dropout)
    self.fc_activation = ReLU()

  def _encode_team(self, team):
    embedding = self.player_embedder(team)
    embedding = self.emb_bn(embedding.moveaxis(-2, -1))
    # #dropout here?
    emnedding = self.regularizer(embedding)

    player_h = self.player_fc(embedding)
    player_h = self.player_fc_bn(player_h)
    player_h = self.fc_activation(player_h)
    player_h = self.regularizer(player_h)
    team_h = player_h.sum(-1)

    for fc, bn in zip(self.team_fc, self.team_fc_bn):
      team_h = fc(team_h)
      team_h = bn(team_h)
      team_h = self.fc_activation(team_h)
      team_h = self.regularizer(team_h)

    return team_h

  def forward(self, home, away):
    home_encoded = self._encode_team(home)
    away_encoded = self._encode_team(away)
    return self.decoder(home_encoded, away_encoded)

## Hyperparameters

In [196]:
home_players = torch.from_numpy(home_player_labels)
away_players = torch.from_numpy(away_player_labels)
assert home_players.max() == away_players.max()
num_players = home_players.max() + 1

embedding_size = 4
player_hidden_size = 12
team_hidden_size = [14, 16]
hidden_sizes = [16]
learning_rate = 1e-3
dropout = 0.5
n_epochs = 75
every = 15
batch_size = 32

dscff_model = DeepSetCatFeedForward(
    num_players, 
    embedding_size, 
    player_hidden_size, 
    team_hidden_size, 
    hidden_sizes,
    dropout=dropout
)
optimizer = torch.optim.Adam(dscff_model.parameters(), lr=learning_rate)
criterion = NLLLoss()
dscff_model

DeepSetCatFeedForward(
  (player_embedder): Embedding(9513, 4)
  (emb_bn): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (player_fc): Conv1d(4, 12, kernel_size=(1,), stride=(1,))
  (player_fc_bn): BatchNorm1d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (team_fc): ModuleList(
    (0): Linear(in_features=12, out_features=14, bias=True)
    (1): Linear(in_features=14, out_features=16, bias=True)
  )
  (team_fc_bn): ModuleList(
    (0): BatchNorm1d(14, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (decoder): CatFeedForward(
    (fc_layers): ModuleList(
      (0): Linear(in_features=32, out_features=16, bias=True)
      (1): Linear(in_features=16, out_features=3, bias=True)
    )
    (bn_layers): ModuleList(
      (0): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (regularizer

## Fitting

In [198]:
for train_set, dev_set, test_set in zip(train_groups, dev_groups, test_groups):
  # batch_size = train_set.shape[0]
  home_train = torch.from_numpy(
      np.stack(train_set['home_lineup_label'].apply(lambda lineup: lineup.split(' - '))).astype(np.int16)
  ).long()
  home_train = home_train.reshape(-1, 11)
  away_train = torch.from_numpy(
      np.stack(train_set['away_lineup_label'].apply(lambda lineup: lineup.split(' - '))).astype(np.int16)
  ).long()
  away_train = away_train.reshape(-1, 11)
  y_train = torch.from_numpy(train_set.loc[:, ['result_label']].values.reshape(-1))
  # y_train = y_train.reshape(-1, batch_size)

  home_dev = torch.from_numpy(
      np.stack(dev_set['home_lineup_label'].apply(lambda lineup: lineup.split(' - '))).astype(np.int16)
  ).long()
  away_dev = torch.from_numpy(
      np.stack(dev_set['away_lineup_label'].apply(lambda lineup: lineup.split(' - '))).astype(np.int16)
  ).long()
  y_dev = torch.from_numpy(dev_set.loc[:, ['result_label']].values.reshape(-1))

  home_test = torch.from_numpy(
      np.stack(test_set['home_lineup_label'].apply(lambda lineup: lineup.split(' - '))).astype(np.int16)
  ).long()
  away_test = torch.from_numpy(
      np.stack(test_set['home_lineup_label'].apply(lambda lineup: lineup.split(' - '))).astype(np.int16)
  ).long()
  y_test = torch.from_numpy(test_set.loc[:, ['result_label']].values.reshape(-1))

  main(
      (home_train, away_train, y_train),
       batch_size,
      (home_dev, away_dev, y_dev),
      (home_test, away_test, y_test),
      np.unique(train_set["league"].values).item(),
      dscff_model, 
      optimizer,
       criterion,
       n_epochs,
       every
  )

Fitting on the Belgium Jupiler League for 75 epochs
............................................................
Initial State
Train Acc%: 47.9365
Dev   Acc%: 42.1818
------------------------------------------------------------
Epoch 15
Avg Train Loss: 0.0311
Train Acc%:     48.0952
Dev   Acc%:     42.1818
------------------------------------------------------------
Epoch 30
Avg Train Loss: 0.0336
Train Acc%:     48.0952
Dev   Acc%:     42.1818
------------------------------------------------------------
Epoch 45
Avg Train Loss: 0.0355
Train Acc%:     48.0952
Dev   Acc%:     42.1818
------------------------------------------------------------
Epoch 60
Avg Train Loss: 0.0332
Train Acc%:     48.0952
Dev   Acc%:     42.1818
------------------------------------------------------------
Epoch 75
Avg Train Loss: 0.0347
Train Acc%:     48.0952
Dev   Acc%:     42.1818
Test Acc%: 47.4638
Fitting on the England Premier League for 75 epochs
.........................................................